In [1]:
import random

from mpi4py import MPI

import numpy as np
import pytest

import basix
import ufl
from basix.ufl import blocked_element, custom_element, element, enriched_element, mixed_element
from dolfinx import default_real_type, default_scalar_type
from dolfinx.fem import (
    Expression,
    Function,
    assemble_scalar,
    create_nonmatching_meshes_interpolation_data,
    form,
    functionspace,
)
from dolfinx.geometry import bb_tree, compute_collisions_points
from dolfinx.mesh import (
    CellType,
    create_mesh,
    create_rectangle,
    create_submesh,
    create_unit_cube,
    create_unit_square,
    locate_entities,
    locate_entities_boundary,
    meshtags,
)


In [2]:
mesh = create_unit_square(MPI.COMM_WORLD, 6, 7)

def left_locator(x):
    return x[0] <= 0.5 + 1e-14

def ref_func(x):
    return x[0] + x[1] ** 2
    
tdim = mesh.topology.dim
cells = locate_entities(mesh, tdim, left_locator)
submesh, sub_to_parent, _, _ = create_submesh(mesh, tdim, cells)

V = functionspace(mesh, ("Lagrange", 2))
u = Function(V)
u.interpolate(ref_func)
    
V_sub = functionspace(submesh, ("DG", 3))
u_sub = Function(V_sub)
    
# Map from parent to sub mesh
u_sub.interpolate(u, cell_map=sub_to_parent)
    
u_sub_exact = Function(V_sub)
u_sub_exact.interpolate(ref_func)
atol = 5 * np.finfo(default_scalar_type).resolution
np.testing.assert_allclose(u_sub_exact.x.array, u_sub.x.array, atol=atol)

# Map from sub to parent
W = functionspace(mesh, ("DG", 4))
w = Function(W)

cell_imap = mesh.topology.index_map(tdim) 
num_cells = cell_imap.size_local + cell_imap.num_ghosts
parent_to_sub = np.full(num_cells, -1, dtype=np.int32)
parent_to_sub[sub_to_parent] = np.arange(len(sub_to_parent))

# Mapping back needs to be restricted to the subset of cells in the submesh
w.interpolate(u_sub_exact, cells=sub_to_parent, cell_map=parent_to_sub)

w_exact = Function(W)
w_exact.interpolate(ref_func, cells=cells)


In [7]:
parent_to_sub

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1,  1,
       -1, -1, -1,  2,  3, -1, -1, -1,  4,  5, -1, -1, -1,  6,  7,  8, -1,
       -1, -1,  9, 10, 11, -1, -1, -1, 12, 13, 14, -1, -1, -1, 15, 16, 17,
       -1, -1, -1, 18, 19, 20, -1, -1, 21, 22, 23, -1, -1, 24, 25, 26, -1,
       27, 28, 29, -1, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41],
      dtype=int32)